In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, median_absolute_error
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.api import VAR
from statsmodels.tsa.statespace.varmax import VARMAX
from scipy.stats import norm
from sklearn.metrics import r2_score

In [8]:
# Laste poll-of-polls data
df = pd.read_csv("../data/all_polls.csv", sep=",", encoding='ISO-8859-1')

In [10]:
# Convert to datetime and set the date to the end of the month
df["Dato"] = pd.to_datetime(df["Dato"])

In [12]:
# Sort values and set index
df = df.sort_values("Dato")
df.set_index("Dato", inplace=True)

In [15]:
df_en=df[["Ap","Høyre","Frp","SV","Sp","KrF","Venstre","MDG","Rødt"]]

In [36]:
model = VAR(df_en)
model_fitted = model.fit(maxlags=3,method = 'ols', trend='n', verbose=True)

C:\Users\jens.nilsen\python\WPy64-31230\python-3.12.3.amd64\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


In [37]:
print(model_fitted.summary())

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Tue, 09, Sep, 2025
Time:                     13:12:06
--------------------------------------------------------------------
No. of Equations:         9.00000    BIC:                  -0.950830
Nobs:                     1860.00    HQIC:                  -1.40691
Log likelihood:          -21954.1    FPE:                   0.187672
AIC:                     -1.67308    Det(Omega_mle):        0.164842
--------------------------------------------------------------------
Results for equation Ap
                coefficient       std. error           t-stat            prob
-----------------------------------------------------------------------------
L1.Ap              0.363292         0.052114            6.971           0.000
L1.Høyre           0.017062         0.051654            0.330           0.741
L1.Frp             0.012630         0.052201            0.242           0.8

In [38]:
# Compute R² for each party
r2_scores = {}
for party in ['Ap', 'Høyre', 'Frp', 'SV', 'Sp', 'KrF', 'Venstre', 'MDG', 'Rødt']:
    r2_scores[party] = r2_score(df_en[party].iloc[model_fitted.k_ar:], model_fitted.fittedvalues[party])

# Print R² values
for party, r2 in r2_scores.items():
    print(f"R² for {party}: {r2:.4f}")

R² for Ap: 0.8995
R² for Høyre: 0.8710
R² for Frp: 0.8774
R² for SV: 0.8043
R² for Sp: 0.9394
R² for KrF: 0.6491
R² for Venstre: 0.4934
R² for MDG: 0.8937
R² for Rødt: 0.8904


In [40]:
model_max = VARMAX(endog=df_en, order=(2,1), trend='n')
model_fitted_max = model_max.fit(disp=True, method='powell', cov_type='robust_approx', maxiter=10)

C:\Users\jens.nilsen\python\WPy64-31230\python-3.12.3.amd64\Lib\site-packages\statsmodels\tsa\statespace\varmax.py:161: EstimationWarning: Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.
  warn('Estimation of VARMA(p,q) models is not generically robust,'
C:\Users\jens.nilsen\python\WPy64-31230\python-3.12.3.amd64\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


LinAlgError: Schur decomposition solver error.